## In class Exercise 4

The purpose of this exercise is to practice topic modeling.
Please use the text corpus you collected in your last in-class-exercise for this exercise.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due tonight November 1st, 2023 at 11:59 PM.
**Late submissions cannot be considered.**

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics.

You may refer the code here:
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
import nltk; nltk.download('stopwords')

!pip install --upgrade spacy

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
#Implemented on a movie dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [ ]:
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#Generating K topics by using LDA and summarizing the topics.

coherence_scores = []
for k in range(2, 11):  # Try different values of K
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=k, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    coherence_scores.append((k, coherence_score))

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]

In [ ]:
print(f"The optimal number of topics is {optimal_k}")
topics = lda_model.show_topics(num_topics=optimal_k, num_words=10, formatted=False)
for topic in topics:
    print(f"Topic {topic[0]}: {' | '.join(word[0] for word in topic[1])}")

The optimal number of topics is 5
Topic 8: evidence | reason | believe | faith | sense | exist | religion | claim | belief | law
Topic 5: ax | max | choose | character | sin | monitor | motif | newsgroup | dn | gift
Topic 9: case | issue | public | group | provide | also | cause | report | patient | number
Topic 6: line | use | system | key | thank | need | file | program | mail | host
Topic 3: write | line | article | say | know | get | think | go | make | organization


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics.

You may refer the code here:
https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

coherence_scores = []

for k in range(2, 11):  # Try different values of K
    lsa_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=k)
    coherence_model = CoherenceModel(model=lsa_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append((k, coherence_score))

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]
print(f"The optimal number of topics is {optimal_k}")

In [ ]:
optimal_lsa_model = LsiModel(corpus=corpus, id2word=id2word, num_topics=optimal_k)

In [ ]:
topics = optimal_lsa_model.show_topics(num_topics=optimal_k, num_words=10, formatted=False)
for topic in topics:
    print(f"Topic {topic[0]}: {' | '.join(word[0] for word in topic[1])}")

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics.

You may refer the code here:
https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install lda2vec

In [ ]:
pip install pyLDAvis

In [ ]:
# Write your code here
import lda2vec
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('/content/drive/MyDrive/ProjectClg/topics.pyldavis.npz', 'r'))
# dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()


In [ ]:
from lda2vec import Lda2vec

coherence_scores = []

for k in range(2, 11):  # Try different values of K
    lda2vec_model = Lda2vec(n_topics=k)
    lda2vec_model.fit(data_lemmatized)

    # Calculate coherence score here (based on your lda2vec implementation)
    coherence_score = lda2vec_model.calculate_coherence()
    # Append coherence_score to coherence_scores list

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]
print(f"The optimal number of topics is {optimal_k}")

In [ ]:
lda2vec_model = Lda2vec(n_topics=optimal_k)
lda2vec_model.fit(data_lemmatized)

In [ ]:
from sklearn.cluster import KMeans

word_vectors = lda2vec_model.word_vectors  # Get word vectors

# Cluster word vectors to identify topics
kmeans = KMeans(n_clusters=optimal_k)
kmeans.fit(word_vectors)
word_labels = kmeans.labels_

# Print top words for each cluster
for topic_id in range(optimal_k):
    words_in_topic = [word for i, word in enumerate(id2word) if word_labels[i] == topic_id]
    print(f"Topic {topic_id}: {', '.join(words_in_topic)}")

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics.

You may refer the code here:
https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
%%capture
!pip install bertopic

In [ ]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [ ]:
from bertopic import BERTopic
# from bertopic import UMAPer
import umap

coherence_scores = []

for k in range(2, 11):  # Try different values of K
    topic_model = BERTopic(language="english", min_topic_size=5, verbose=True)
    topics, _ = topic_model.fit_transform(docs)
    umap_model = umap.UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
    umap_embeddings = umap_model.fit_transform(topics)
    coherence_score = umap_model.score(umap_embeddings)
    coherence_scores.append((k, coherence_score))

# Find the K with the highest coherence score
optimal_k = max(coherence_scores, key=lambda x: x[1])[0]
print(f"The optimal number of topics is {optimal_k}")


In [ ]:
topic_model = BERTopic(language="english", min_topic_size=5, verbose=True)
topics, _ = topic_model.fit_transform(data_lemmatized)

In [ ]:
for topic_id, words in topic_model.get_topics().items():
    print(f"Topic {topic_id}: {', '.join(words)}")

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

Follow the guidelines from the essay to enhance your explanation:

* Writing logic

  Pay attention to how you express your thoughts. For example:

  * Weak Writing Logic: “Artificial Intelligence is risky because it is new technology.”

  * Strong Writing Logic: “Artificial Intelligence presents ethical risks such as data privacy concerns and algorithmic bias, which necessitate cautious implementation and regulation.”

* Topic of sentences

  * Focus and Direction: It provides a focus and sets the direction for the paragraph, ensuring that the reader knows what to expect.
  * Reader Guidance: It serves as a guidepost for the reader, making it easier to follow the flow of ideas and arguments in the document.
  * Support for Thesis: In academic papers, topic sentences help in elaborating or providing evidence for the thesis statement or research question.

* Writing flow

  * Transition: Smooth and logical transitions between sentences, paragraphs, and sections.
  * Rhythm: Variation in sentence length and structure to maintain reader engagement.
  * Sequence: The order of points or arguments contributes to a smooth reading experience.
  For example:
    * Weak Writing Flow: “We studied machine learning algorithms. Ethics are important. Data was collected.”
    * Strong Writing Flow: “We initiated our study by focusing on machine learning algorithms. Recognizing the ethical implications, we carefully curated our data set.”

**ANSWER:**

It's critical to evaluate the advantages and disadvantages of the four topic modeling algorithms—Latent Dirichlet Allocation (LDA), Latent Semantic Analysis (LSA), lda2vec, and BERTopic—in order to ascertain which is more appropriate for particular use situations.

Due to its interpretability, LDA is a well-established and conventional technique that is preferred. It is perfect for tasks where subject comprehension is crucial since it generates topics that are readable by humans. It can be difficult to precisely estimate the number of topics in advance, though, as LDA demands you to do.

Another traditional approach, LSA, is excellent at reducing dimensionality and capturing the semantic links between concepts. It is appropriate for jobs where capturing semantic similarity is more important than subject interpretability, even though it might not provide easily interpretable topics.

A new method called lda2vec finds a balance between semantic comprehension and interpretability. It makes use of word embeddings, which are capable of capturing intricate semantic associations. Although lda2vec is designed to offer more interpretable subjects than LSA, it can be computationally demanding and difficult to implement.

In contrast, BERTopic is predicated on BERT embeddings, a cutting-edge language model. Because of BERT's superior language understanding abilities, it usually generates topics of a high caliber. BERTopic's capacity to automatically calculate the number of topics reduces the need for manual hyperparameter adjustment, which is one of its significant advantages. Because of this, it's perfect for jobs where output is crucial.

The computing demands of the BERT topic, however, can be high during both training and inference. Additionally, because it tends to use a more "black-box" approach, it might not offer the same amount of interpretability as LDA.

In conclusion, your unique use case and priorities will determine which topic modeling algorithm is appropriate for you. LDA is a good option if the number of topics is known ahead of time and interpretability is your primary priority. Tasks that prioritize collecting semantic linkages are a good fit for LSA. Although it requires more complicated implementation, lda2vec offers a balance between interpretability and semantic understanding. Even if some interpretability must be given up, BERTopic's fast performance and automatic topic number selection make it the perfect choice for applications where results are crucial. In the end, the decision should be in line with the goals and trade-offs of your particular use case.